In [1]:
import random

import matplotlib.animation as animation
from matplotlib.widgets import Slider, Button
import matplotlib as mpl
from matplotlib import pyplot as plt
import scipy.interpolate as inter
import numpy as np
import pandas as pd

from wsp import util
from wsp import ds
from wsp import tsp

QTREE = ds.PKPRQuadTree

In [2]:
data = [] # num of jumps, left points, right points
# MARK: SETUP
N_LEFT  = 9 # How many points in the left cluster
N_RIGHT = 1 # How many points in the right cluster
RADIUS_LEFT = 1.0
RADIUS_RIGHT = 1.0
OFFSET_RIGHT = 10.0

def calculate_jumps(old_path : list[ds.Point], new_path : list[ds.Point]) -> int:
    jumps = 0

    old_edges = {
        frozenset([old_path[i], old_path[i + 1]])
        for i in range(len(old_path) - 1)
    }
    new_edges = {
        frozenset([new_path[i], new_path[i + 1]])
        for i in range(len(new_path) - 1)
    }
    # print(old_edges - new_edges)

    return len(old_edges - new_edges)

In [3]:
def run_simulation() -> list:
    left_points : list[ds.Point] = util.generate_points(N_LEFT, lambda: (random.uniform(-RADIUS_LEFT, RADIUS_LEFT), random.uniform(-RADIUS_LEFT, RADIUS_LEFT))) # Generate points
    right_points : list[ds.Point] = util.generate_points(N_RIGHT, lambda: (random.uniform(-RADIUS_RIGHT, RADIUS_RIGHT) + OFFSET_RIGHT, random.uniform(-RADIUS_RIGHT, RADIUS_RIGHT)))
    # left_points = [ds.Point(0,1), ds.Point(0,-1), ds.Point(1,1), ds.Point(1,-1), ds.Point(0.25,0.25), ds.Point(1,0)]
    # right_points = [ds.Point(10,0)+ ds.Point(10,0.5)]

    ts_problem = tsp.TravellingSalesmanProblem[QTREE](QTREE, right_points + left_points, np.array([None, None]), s=2.0)
    left_problem = tsp.TravellingSalesmanProblem[QTREE](QTREE, left_points, np.array([None, None]), s=2.0)

    first_right_index = next(i for i in range(500) if ts_problem.dp_path[0][i] in right_points)
    

    new_solve_small = ts_problem.dp_path[0][first_right_index + N_RIGHT:]
    new_solve_small.extend(# REVIEW: we may not actually need this could save some time
        x
        for x in ts_problem.dp_path[0][:first_right_index]
        if x not in new_solve_small
    )
    new_solve_small = new_solve_small[:-1] if first_right_index == 0 else new_solve_small

    # ensure that the left points are all in a row
    # assert set(new_solve_small) == set(ts_problem.dp_path[0]) - set(ts_problem.dp_path[0][first_right_index:first_right_index + N_RIGHT - 1])
    assert len(new_solve_small) == len(left_problem.dp_path[0]) - 1
    jumps = calculate_jumps(left_problem.dp_path[0], new_solve_small)

    return jumps, left_points, right_points

# run_simulation()

In [5]:
for _ in range(100):
    data.append(run_simulation())
len(data)

1070

In [6]:
df = pd.DataFrame(data, columns=["jumps", "left", "right"])
df

,jumps,left,right
0,1,"[P(0.4938, -0.2621), P(0.9095, 0.6923), P(-0.7...","[P(10.2014, -0.0578)]"
1,1,"[P(-0.2790, 0.6679), P(0.3335, 0.0579), P(-0.2...","[P(10.8157, 0.1497)]"
2,1,"[P(0.8106, 0.5171), P(-0.1068, 0.6870), P(0.17...","[P(10.9450, -0.2778)]"
3,1,"[P(0.7309, -0.6175), P(-0.5671, 0.5256), P(-0....","[P(9.7636, 0.7032)]"
4,2,"[P(0.1085, -0.1883), P(-0.3431, -0.8688), P(-0...","[P(10.5275, 0.0144)]"
...,...,...,...
1065,1,"[P(0.1728, -0.3373), P(-0.5097, 0.5195), P(-0....","[P(9.2474, 0.6454)]"
1066,3,"[P(0.5468, -0.4888), P(0.6910, -0.9430), P(-0....","[P(9.3696, 0.8399)]"
1067,3,"[P(-0.0055, 0.5235), P(0.8618, -0.4858), P(0.7...","[P(9.4457, -0.9193)]"
1068,5,"[P(0.5112, -0.3121), P(-0.1051, -0.0071), P(0....","[P(10.9702, -0.9170)]"


In [7]:
df["jumps"].value_counts()

jumps
1    627
2    242
3    147
4     39
5     13
6      2
Name: count, dtype: int64